In [1]:
import pandas as pd
import numpy as np
import re

In [3]:
df = pd.read_csv('../data/all_data.csv')

In [69]:
def clean_string(query):
    return re.sub('\s+', ' ', query.replace('(', ' ').replace(')', ' ').replace(',', ' , ')).strip()

In [7]:
benign = df.loc[df.label == 0 ].loc[df.is_obf == 0]
malicious = df.loc[df.label == 1].loc[df.is_obf == 0]
malicious

,payload,length,is_obf,label
436,1' where 6406=6406;select count * from rdb$fie...,115,0,1
437,1 and 8514= select count * from domain.domains...,111,0,1
438,-3136%' or 3400=6002,21,0,1
439,1 where 7956=7956 or sleep 5 #,31,0,1
440,-7387' order by 1--,22,0,1
...,...,...,...,...
30151,"1' and extractvalue 7982 , concat 0x5c , 0x717...",111,0,1
30152,"1"" order by 1--",15,0,1
30153,"1' procedure analyse extractvalue 5840 , conca...",149,0,1
30154,1 rlike select * from select sleep 5 sgvo -- saxs,53,0,1


In [99]:
data = benign.append(malicious.iloc[0])

In [94]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse

count_vect = CountVectorizer(analyzer='word', ngram_range=(3,3))
raw_counts = count_vect.fit_transform(data['payload'])

# Append metadata to sparse matrix that will be used to train the classifier
num_feats = data[['length']].values
train_data = sparse.hstack((raw_counts, num_feats))

In [101]:
from sklearn.ensemble import IsolationForest

score = IsolationForest(random_state = 42).fit(train_data).decision_function(train_data)
ranked_index = np.argsort(score) 
malicious_rank = ranked_index.tolist().index(19304)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


In [103]:
malicious_rank

135

In [109]:
import warnings
warnings.filterwarnings('ignore')

samples = 15

dataset = [benign.append(malicious.iloc[i]) for i in range(samples)]
count_vect = CountVectorizer(analyzer='word', ngram_range=(3,3))
raw_counts = [count_vect.fit_transform(data['payload']) for data in dataset]

# Append metadata to sparse matrix that will be used to train the classifier
lengths = [data[['length']].values for data in dataset]
train_data = [sparse.hstack((raw_counts[i], lengths[i])) for i in range(samples)]
scores = [IsolationForest(random_state = 42).fit(data).decision_function(data) for data in train_data]
ranked_indexes = [np.argsort(score) for score in scores]
malicious_rank = [l.tolist().index(19304) for l in ranked_indexes]
print('Anamoly rank of the malicious datapoint:')
for i in range(15):
    print('datapoint %d: %d' % (i, malicious_rank[i]))

print('Average rank = %d' % (sum(malicious_rank)/len(malicious_rank)))

Anamoly rank of the malicious datapoint:
datapoint 0: 135
datapoint 1: 18
datapoint 2: 4890
datapoint 3: 1439
datapoint 4: 3048
datapoint 5: 20
datapoint 6: 37
datapoint 7: 43
datapoint 8: 43
datapoint 9: 57
datapoint 10: 388
datapoint 11: 128
datapoint 12: 647
datapoint 13: 64
datapoint 14: 19
Average rank = 731


In [113]:
import warnings
warnings.filterwarnings('ignore')

samples = 100

dataset = [benign.append(malicious.iloc[i]) for i in range(samples)]
count_vect = CountVectorizer(analyzer='word', ngram_range=(3,3))
raw_counts = [count_vect.fit_transform(data['payload']) for data in dataset]

# Append metadata to sparse matrix that will be used to train the classifier
lengths = [data[['length']].values for data in dataset]
train_data = [sparse.hstack((raw_counts[i], lengths[i])) for i in range(samples)]
scores = [IsolationForest(random_state = 42).fit(data).decision_function(data) for data in train_data]
ranked_indexes = [np.argsort(score) for score in scores]
malicious_rank = [l.tolist().index(19304) for l in ranked_indexes]
print('Anamoly rank of the malicious datapoint:')
for i in range(15):
    print('datapoint %d: %d' % (i, malicious_rank[i]))

print('Average rank = %d' % (sum(malicious_rank)/len(malicious_rank)))

Anamoly rank of the malicious datapoint:
datapoint 0: 135
datapoint 1: 18
datapoint 2: 4890
datapoint 3: 1439
datapoint 4: 3048
datapoint 5: 20
datapoint 6: 37
datapoint 7: 43
datapoint 8: 43
datapoint 9: 57
datapoint 10: 388
datapoint 11: 128
datapoint 12: 647
datapoint 13: 64
datapoint 14: 19
Average rank = 505


In [114]:
np.sort(malicious_rank)

array([    3,    10,    10,    10,    11,    11,    14,    14,    14,
          15,    15,    16,    16,    16,    17,    18,    18,    19,
          19,    20,    21,    21,    22,    23,    23,    23,    24,
          26,    26,    27,    27,    27,    29,    29,    30,    31,
          35,    37,    38,    40,    42,    42,    42,    42,    43,
          43,    43,    48,    48,    48,    48,    54,    57,    64,
          66,    73,    73,    74,    76,    78,    80,    82,   104,
         108,   114,   128,   135,   140,   144,   146,   146,   161,
         175,   178,   201,   224,   266,   292,   335,   388,   389,
         414,   416,   452,   499,   506,   539,   647,   695,   787,
        1328,  1439,  1439,  1452,  1932,  2260,  3048,  4890,  4890,
       17067])